In [4]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy, os
from cartopy import feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy import crs as ccrs
from erddapy import ERDDAP
import datetime as dt
import json
import numpy as np
from cmocean import cm
import cmocean

# from geopy.distance import vincenty
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

__Figure for MBON propsal__

Data sources to illustrate:

    - Rockfish Cruise stations
    - Calcofi
	- Glider lines
	- Wind/Wave buoys
	- ATN tracks
	- HFR
	- Chl-a
	- SST
	- eDNA - ships, LRAUV tracks, point sample

In [5]:
# Mapping helper function
def make_map(projection=ccrs.PlateCarree(),extent=(-130.5,-116.75,32,42)):
    fig, ax = plt.subplots(
        subplot_kw={'projection': projection},
        figsize=(12, 15)
    )
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    gl = ax.gridlines(draw_labels=True,)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlabel_style = {'size': 22}
    gl.ylabel_style = {'size': 22}
    gl.ylocator = mticker.FixedLocator(range(32,48,4))
    gl.xlocator = mticker.FixedLocator(range(-128,-100,4))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    land_10m  = cfeature.NaturalEarthFeature(category='physical', name='land',
                            scale='10m', facecolor=cfeature.COLORS['land'], edgecolor='k')
    ax.add_feature(land_10m,zorder=20)
#     ocean_50m  = cfeature.NaturalEarthFeature(category='physical', name='ocean',
#                             scale='50m', facecolor=cfeature.COLORS['water'])
#     ax.add_feature(ocean_50m,zorder=0)
    return fig, ax

## Rockfish Cruise station data

In [6]:
## Access the rockfish cruise data from ERDDAP
e = ERDDAP(
    server='https://coastwatch.pfeg.noaa.gov/erddap',
    protocol='tabledap',
)

dataset_id = "FED_Rockfish_Catch"

variables = [
 'time',
 'latitude',
 'longitude',
 'cruise',
 'station'
]
e.dataset_id = dataset_id
e.variables = variables

rf_df = e.to_pandas(
    index_col='time (UTC)',
    parse_dates=True,
).dropna()
rf_df = rf_df.tz_convert(None)

In [7]:
rf_2015 = rf_df[rf_df.index > dt.datetime(2015,1,1)]
rf_2015['time_UTC'] = rf_2015.index
rf_2015.groupby(['cruise','station']).first().reset_index().to_csv('FED_rockfish_Cruise.csv',index=False)

/Users/patrick/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
rf_2015.groupby(['cruise','station']).first().reset_index()

,cruise,station,latitude (degrees_north),longitude (degrees_east),time_UTC
0,1505,101,36.3027,-121.9515,2015-06-11 04:25:33
1,1505,103,36.3040,-122.0200,2015-06-11 05:33:49
2,1505,104,36.3062,-122.1002,2015-06-11 06:57:16
3,1505,105,36.3098,-122.1975,2015-06-11 08:36:46
4,1505,106,36.3042,-122.3060,2015-06-11 10:18:22
...,...,...,...,...,...
293,1802,617,40.9787,-124.9200,2018-05-20 10:47:00
294,1802,621,41.4812,-124.2735,2018-05-21 04:59:39
295,1802,622,41.4890,-124.3528,2018-05-21 06:21:14
296,1802,623,41.4847,-124.4657,2018-05-21 07:55:59


### Calcofi Stataion

In [6]:
e = ERDDAP(
server='https://coastwatch.pfeg.noaa.gov/erddap',
protocol='tabledap',
)

dataset_id = "erdCalCOFIstns"

variables = [
 'time',
 'latitude',
 'longitude',
 'order_occupied',
]
e.dataset_id = dataset_id
e.variables = variables

df_cal = e.to_pandas(
    index_col='time (UTC)',
    parse_dates=True,
).dropna()
df_cal['dateTime'] = df_cal.index
df_cal = df_cal[df_cal["dateTime"].dt.year ==2015]


In [7]:
df_c = df_cal[(df_cal['order_occupied']> 30) & (df_cal['order_occupied']< 100)]
llat = df_c["latitude (degrees_north)"].values
llon = df_c["longitude (degrees_east)"].values

In [8]:
stations = [70,56,57,58,59,60,61]
df_c = df_cal[(df_cal['order_occupied'].isin(stations)) & (df_cal["latitude (degrees_north)"] < 36)]

In [9]:
df_c.to_csv('../Data/CalCOFI_Cruise.csv',index=False)

In [3]:
station_txt = [str(st) for st in df_c['order_occupied'].values]
fig,ax = make_map()
ax.scatter(df_c["longitude (degrees_east)"],df_c["latitude (degrees_north)"])

NameError: name 'df_c' is not defined

In [2]:
for col in df_c.columns:
    print(col)

NameError: name 'df_c' is not defined

## Coastwatch Global Seascape Data

Global seascape prototype data is descibed as: "Biogeographic framework. Space and time classified simultaneously from synoptic time series using hierarchical and topology preserving machine learning"

Availible in portal at: http://cwcgom.aoml.noaa.gov/cgom/OceanViewer/#

THREDDS: http://cwcgom.aoml.noaa.gov/thredds/dodsC/SEASCAPE_MONTH/SEASCAPES.nc.html

In [8]:
thredds_url = "http://cwcgom.aoml.noaa.gov/thredds/dodsC/SEASCAPE_MONTH/SEASCAPES.nc"
ss_ds = xr.open_dataset(thredds_url)
# sub_domain = ss_ds.sel(lat=slice(32,50),lon=slice(-132,-116))
sub_domain = ss_ds.sel(lat=slice(32,50),lon=slice(-132,-116)) # Central Coast
xx,yy = np.meshgrid(sub_domain['lon'], sub_domain['lat'])

In [ ]:
seasonal = sub_domain['CLASS'].isel(time=220).values
flat = seasonal.flatten()
ix = np.where(np.isfinite(flat))
cat_values = np.unique(flat[ix[0][:]])
cat_key = np.arange(0,len(cat_values),dtype=int)

for i in cat_key:
    ix = np.where(seasonal == cat_values[i])
    seasonal[ix] = i

ticks = (np.arange(len(cat_values)) + 0.5)*(len(cat_values)-1)/len(cat_values)

fig, ax = make_map()

im = ax.pcolormesh(xx, yy, seasonal,transform=ccrs.PlateCarree(),cmap=plt.cm.get_cmap('cmo.speed', len(cat_key)))
# im = ax.pcolormesh(xx, yy, seasonal,transform=ccrs.PlateCarree(),cmap='cmo.amp')
divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=.45, axes_class=plt.Axes)
fig.add_axes(ax_cb)
formatter = plt.FuncFormatter(lambda val, loc: int(cat_values[int(val)]))
formatter = plt.FuncFormatter(lambda val, loc: int(cat_values[np.where(ticks == val)][0]))

cbar = plt.colorbar(im, cax=ax_cb, ticks=ticks,format=formatter)
cbar.ax.tick_params(labelsize=20) 

## Trinidad Head Line

In [ ]:
fname = './trinidad_head_station_locations.csv'
df_thl = pd.read_csv(fname)
df_thl

In [ ]:
# fig, ax = make_map()
# ax.scatter(df_thl['Lon'], df_thl['Lat'],marker='d',c='k')

## HFR 6-km radar data

In [ ]:
# hfr_url = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd"
# hfr_ds = xr.open_dataset(hfr_url)
# recent_ds = hfr_ds.isel(time=-2)
# u = recent_ds['u']
# v = recent_ds['v']
# speed = np.sqrt(u.values**2 + v.values**2)
# xx,yy = np.meshgrid(recent_ds['lon'], recent_ds['lat'])

In [ ]:
# ix = np.where(speed < .5)

In [ ]:
# fig, ax = make_map()
# ax.quiver(xx[ix],yy[ix],u.values[ix],v.values[ix],speed[ix],
#           cmap=cm.speed,
#           scale_units='xy',
#           scale=.5,)
# # ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'], marker='.',c='k')

## Get Glider lines

In [ ]:
e = ERDDAP(
    server='https://data.ioos.us/gliders/erddap',
    protocol='tabledap',
)
min_lon,max_lon,min_lat,max_lat= (-130,-116.75,32,42)
min_time = "2018-01-01T00:00:00Z"
max_time = "2019-01-01T00:00:00Z"
kw = {
    'standard_name': 'sea_water_temperature',
    'min_lon': min_lon,
    'max_lon': max_lon,
    'min_lat': min_lat,
    'max_lat': max_lat,
    'min_time': min_time,
    'max_time': max_time,
    'cdm_data_type': 'trajectoryprofile'
}
search_url = e.get_search_url(response='csv', **kw)
search = pd.read_csv(search_url)
gliders = search['Dataset ID'].values

In [ ]:
gliders

In [ ]:
e.variables = variables
variables = [
 'latitude',
 'longitude',
 'time',
]
gliders_to_plot = ["sp058-20180126T1753",'UW157-20180417T1832',"Nemesis-20180830T0000","sp025-20180430T1727"]
for i, glider_id in enumerate(gliders_to_plot):
    e.dataset_id = glider_id
    df = e.to_pandas(
        index_col='time (UTC)',
        parse_dates=True,
    ).dropna()
    df['glider_name'] = glider_id
    print("Pulling glider: {0}".format(glider_id))
    if i == 0:
        df_gliders = df
    else:
        df_array = (df_gliders, df)
        df_gliders = pd.concat(df_array)

In [ ]:
e.variables

In [ ]:
gliders_to_plot = ["sp058-20180126T1753",'UW157-20180417T1832',"Nemesis-20180830T0000","sp025-20180430T1727"]
fig, ax = make_map()
# ax.quiver(xx,yy,u,v,speed,cmap=cm.speed,units='x',scale=3,scale_units='x') # HFR
# for i, name in enumerate(df_gliders['glider_name'].unique()):
for i, name in enumerate(gliders_to_plot):
    df_temp = df_gliders[df_gliders['glider_name']==name]
    ax.scatter(df_temp['longitude (degrees_east)'], df_temp['latitude (degrees_north)'],s=20,marker='.',c='k') # Gliders
ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'], marker='.') # Rockfish


### Coastal Relief Model

In [ ]:
fname = './crm.nc'
ds_bathy = xr.open_dataset(fname)
elev = ds_bathy['Band1'] * -1
llon, llat = np.meshgrid(ds_bathy['lon'], ds_bathy['lat'])
levels = np.append(np.arange(-10,501,100), np.array((1000,1500,2000,3000,5000)))

## ACCESS Trawl stations

In [ ]:
# Mapping helper function
def make_map_inlet(projection=ccrs.PlateCarree(),extent=(-123.75,-122.25,36.9,38.75)):
    fig, ax = plt.subplots(
        subplot_kw={'projection': projection},
        figsize=(6, 6)
    )
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    gl = ax.gridlines(draw_labels=True,)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlabel_style = {'size': 22}
    gl.ylabel_style = {'size': 22}
    gl.ylocator = mticker.FixedLocator(range(32,48,10))
    gl.xlocator = mticker.FixedLocator(range(-128,-100,10))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    land_10m  = cfeature.NaturalEarthFeature(category='physical', name='land',
                            scale='10m', facecolor=cfeature.COLORS['land'], edgecolor='k',linewidths=2)
    ax.add_feature(land_10m,)
#     ocean_50m  = cfeature.NaturalEarthFeature(category='physical', name='ocean',
#                             scale='50m', facecolor=cfeature.COLORS['water'])
#     ax.add_feature(ocean_50m,zorder=0)
    return fig, ax

# Mapping helper function
def make_map_inlet_humboldt(projection=ccrs.PlateCarree(),extent=(-125.85,-124,40.325,41.75)):
    fig, ax = plt.subplots(
        subplot_kw={'projection': projection},
#         figsize=(6, 6)
    )
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    gl = ax.gridlines(draw_labels=True,)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlabel_style = {'size': 22}
    gl.ylabel_style = {'size': 22}
    gl.ylocator = mticker.FixedLocator(range(32,48,4))
    gl.xlocator = mticker.FixedLocator(range(-128,-100,4))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    land_10m  = cfeature.NaturalEarthFeature(category='physical', name='land',
                            scale='10m', facecolor=cfeature.COLORS['land'], edgecolor='k',linewidths=2)
    ax.add_feature(land_10m,)
#     ocean_50m  = cfeature.NaturalEarthFeature(category='physical', name='ocean',
#                             scale='50m', facecolor=cfeature.COLORS['water'])
#     ax.add_feature(ocean_50m,zorder=0)
    return fig, ax

In [ ]:
fname = "access_trawl.csv"
df_access = pd.read_csv(fname)
df_access['eventDate'] = pd.to_datetime(df_access['eventDate'])
station = df_access.groupby('eventDate').first()
station = station[station.index > dt.datetime(2013,1,1)]
station.to_csv('ACCESS_Cruise.csv')

In [ ]:
fname = "access_trawl.csv"
df_access = pd.read_csv(fname)
df_access['eventDate'] = pd.to_datetime(df_access['eventDate'])
station = df_access.groupby('eventDate').first()
station = station[station.index > dt.datetime(2013,1,1)]
fig, ax = make_map_inlet(extent=(-123.2,-121.22,35.5,37.675))
im = ax.pcolormesh(xx, yy, seasonal,transform=ccrs.PlateCarree(),cmap=plt.cm.get_cmap('cmo.speed', len(cat_key)))
plt.FuncFormatter(lambda val, loc: int(cat_values[np.where(ticks == val)][0]))
# fname = '/Users/patrick/Documents/CeNCOOS/store-locator/data/sanctuary_files/gfnms_py2/GFNMS_py.shp'
# GF_shape = Reader(fname)
# ax.add_geometries(GF_shape.geometries(),ccrs.PlateCarree(),edgecolor='k',alpha=.85,lw=2,zorder=1)
fname = '/Users/patrick/Documents/CeNCOOS/store-locator/data/sanctuary_files/mbnms_py2/mbnms_py.shp'
GF_shape = Reader(fname)
ax.add_geometries(GF_shape.geometries(),ccrs.PlateCarree(),edgecolor='#c51b8a',alpha=.85,facecolor="None",linestyle='dashed',lw=3,zorder=1)
# fname = '/Users/patrick/Documents/CeNCOOS/store-locator/data/sanctuary_files/cbnms_py2/CBNMS_py.shp'
# GF_shape = Reader(fname)
# ax.add_geometries(GF_shape.geometries(),ccrs.PlateCarree(),facecolor='#756bb1',alpha=.85,edgecolor="k",lw=2,zorder=1)

cbar = plt.colorbar(im, cax=ax_cb, ticks=ticks,format=formatter)
cbar.ax.tick_params(labelsize=20) 

# CS = ax.contourf(llon,llat,elev,levels,alpha=.75,cmap=cm.deep)
ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'],s=110,edgecolor='w', marker='D',c='#4393c3',zorder=4) # Rockfish
# ax.scatter(station['Lon'],station['Lat'],c='#d6604d',edgecolor='w',s=210,marker='o',linewidths=2,zorder=4) # ACCESS

plot_seal_tracks(ax, atn_df)
# ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'], marker='D',edgecolor='w') # Rockfish
ax.scatter(df_thl['Lon'][:-3], df_thl['Lat'][:-3],marker='s',edgecolor='w',linewidths=2,c='#f46d43',s=115)
ax.scatter(df_c['longitude (degrees_east)'], df_c['latitude (degrees_north)'],c=".25", marker='v',linewidths=3,edgecolor='#d6604d', s=135,zorder=50) # calcofi
# MBARI TS Stations

ax.scatter([-122.022,-122.378,-121.847,-122.772,-123.485,-124.195, -124.902],[36.747,36.697,36.797,36.453,36.120,35.786,35.453],marker='P',s=170,zorder=21,linewidths=2,edgecolor='w',c="#ff7700")

for i, name in enumerate(gliders_to_plot):
    df_temp = df_gliders[df_gliders['glider_name']==name]
    ax.scatter(df_temp['longitude (degrees_east)'], df_temp['latitude (degrees_north)'],s=75,marker='.',c='k',zorder=2) # Gliders

plt.savefig('Centeral_CA_insert.png',dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
lon = [-122.022,-122.378,-121.847,-122.772,-123.485,-124.195, -124.902]

lat = [36.747,36.697,36.797,36.453,36.120,35.786,35.453]
mbari_df = pd.DataFrame(data={'latitude (degrees_north)':lat,'longitude (degrees_east)':lon})
mbari_df['source'] = 'mbari'

In [ ]:
# df_thl = df_thl.rename(columns={'Lat':'latitude (degrees_north)',"Lon":'longitude (degrees_east)'})
df_thl['source'] = 'trinidad_head_line'
df_all = pd.concat((mbari_df,df_thl[['latitude (degrees_north)','longitude (degrees_east)','source']]))
df_c['source'] = 'CalCOFI_2015'
df_all = pd.concat((df_all,df_c[['latitude (degrees_north)','longitude (degrees_east)','source']])).reset_index(drop=True)
rf_2015['source'] = 'rockfish_2015'
rf_2015 = rf_2015.groupby(['cruise','station']).first().reset_index()
df_all = pd.concat((df_all,rf_2015[['latitude (degrees_north)','longitude (degrees_east)','source']])).reset_index(drop=True)
access_df = station.rename(columns={'Lat':'latitude (degrees_north)',"Lon":'longitude (degrees_east)'})
access_df = access_df.reset_index(drop=True)
access_df['source'] = 'ACCESS_2013_2014'
df_all = pd.concat((df_all,access_df[['latitude (degrees_north)','longitude (degrees_east)','source']])).reset_index(drop=True)
df_all.to_csv('MBON-proposal-map-coords.csv',index=False)

In [ ]:
fname = './etopo1.nc'
ds_bathy = xr.open_dataset(fname)
elev = ds_bathy['Band1'] * -1
llon, llat = np.meshgrid(ds_bathy['lon'], ds_bathy['lat'])
levels = np.append(np.arange(-10,501,100), np.array((1000,1500,2000,3000,16000)))


In [ ]:
fname = './trinidad_head_station_locations.csv'
df_thl = pd.read_csv(fname)

fig, ax = make_map_inlet_humboldt()
im = ax.pcolormesh(xx, yy, seasonal,transform=ccrs.PlateCarree(),cmap=plt.cm.get_cmap('cmo.speed', len(cat_key)))
# im = ax.pcolormesh(xx, yy, seasonal,transform=ccrs.PlateCarree(),cmap='cmo.amp')
# divider = make_axes_locatable(ax)
# ax_cb = divider.new_horizontal(size="5%", pad=.45, axes_class=plt.Axes)
# fig.add_axes(ax_cb)
# formatter = plt.FuncFormatter(lambda val, loc: int(cat_values[int(val)]))
# formatter = plt.FuncFormatter(lambda val, loc: int(cat_values[np.where(ticks == val)][0]))

# cbar = plt.colorbar(im, cax=ax_cb, ticks=ticks,format=formatter)
# cbar.ax.tick_params(labelsize=20) 
# CS = ax.contourf(llon,llat,elev,levels,alpha=.75,cmap=cm.deep)
for i, name in enumerate(gliders_to_plot):
    df_temp = df_gliders[df_gliders['glider_name']==name]
    ax.scatter(df_temp['longitude (degrees_east)'], df_temp['latitude (degrees_north)'],s=75,marker='.',c='k',zorder=1) # Gliders

ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'],s=90, marker='D',edgecolor='w',zorder=2) # Rockfish

ax.scatter(df_thl['Lon'][:-3], df_thl['Lat'][:-3],marker='s',edgecolor='w',linewidths=2,c='#f46d43',s=115,zorder=2)

fname = '/Users/patrick/Documents/CeNCOOS/store-locator/data/boem_call_areas_10_01_18/BOEM Call_Areas_10_01_18.shp'
GF_shape = Reader(fname)
ax.add_geometries(GF_shape.geometries(),ccrs.UTM(zone=10),facecolor='#bdbdbd',alpha=.85,edgecolor="k",lw=2,zorder=1,hatch='//')


plt.savefig('trinidad_line_insert.png',dpi=300, bbox_inches='tight', pad_inches=0)

### Add ATN Tracks

In [ ]:
e = ERDDAP(
    server='https://coastwatch.pfeg.noaa.gov/erddap',
    protocol='tabledap',
)
e.dataset_id = "gtoppAT"

# e.constraints = {
#     'latitude>=': min_lat,
#     'latitude<=': max_lat,
#     'longitude>=': 360 + min_lon,
#     'longitude<=': 360 + max_lon,
# }

e.variables = [
 'time',
 'latitude',
 'longitude',
 'toppID',
 'commonName',
 'yearDeployed',
 'isDrifter'
]

atn_df = e.to_pandas().dropna()

print(atn_df['commonName'].unique())
atn_df['longitude (degrees_east)'] = -360+atn_df['longitude (degrees_east)']

In [ ]:
def plot_seal_tracks(ax, atn_df):
    """ Plot the atn elephan seal tracks deployed after 2016"""
    elph_seals = atn_df[(atn_df['commonName'] == "Northern Elephant Seal")&(atn_df['yearDeployed'] >= 2016)]
    for t_id in elph_seals['toppID'].unique()[:8]:
        tag = elph_seals[(elph_seals['toppID'] == t_id)]
        ax.plot(tag['longitude (degrees_east)'], tag['latitude (degrees_north)'], marker="o",lw=2,ms=7,mew=1.5,mec='.95',c="#7F7F7F")

In [ ]:
# elph_seals = atn_df[(atn_df['commonName'] == "Northern Elephant Seal")&(atn_df['yearDeployed'] >= 2016)]
# elph_seals['toppID'].un

In [ ]:
gliders_to_plot = ["sp058-20180126T1753",'UW157-20180417T1832',"Nemesis-20180830T0000","sp025-20180430T1727"]
fig, ax = make_map()
im = ax.pcolormesh(xx, yy, seasonal,transform=ccrs.PlateCarree(),cmap=plt.cm.get_cmap('cmo.speed', len(cat_key)))
# im = ax.pcolormesh(xx, yy, seasonal,transform=ccrs.PlateCarree(),cmap='cmo.amp')
divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=.45, axes_class=plt.Axes)
fig.add_axes(ax_cb)
formatter = plt.FuncFormatter(lambda val, loc: int(cat_values[int(val)]))
formatter = plt.FuncFormatter(lambda val, loc: int(cat_values[np.where(ticks == val)][0]))

cbar = plt.colorbar(im, cax=ax_cb, ticks=ticks,format=formatter)
cbar.ax.tick_params(labelsize=20) 

for i, name in enumerate(gliders_to_plot):
    df_temp = df_gliders[df_gliders['glider_name']==name]
    ax.scatter(df_temp['longitude (degrees_east)'], df_temp['latitude (degrees_north)'],s=75,marker='.',c='k') # Gliders

    
    
# temp_df = rf_2015[(rf_2015['station'] < 200) & (rf_2015['station'] < 200)]   
# ax.scatter(temp_df['longitude (degrees_east)'], temp_df['latitude (degrees_north)'], marker='.') # Rockfish
plot_seal_tracks(ax, atn_df)

ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'], marker='D',edgecolor='w') # Rockfish
ax.scatter(df_thl['Lon'][:-3], df_thl['Lat'][:-3],marker='s',edgecolor='w',linewidths=2,c='#f46d43',s=115)

ax.scatter(df_c['longitude (degrees_east)'], df_c['latitude (degrees_north)'],c=".25", marker='v',linewidths=3,edgecolor='#d6604d', s=135,zorder=50) # calcofi
ax.scatter([-122.022,-122.378,-121.847,-122.772,-123.485,-124.195, -124.902],[36.747,36.697,36.797,36.453,36.120,35.786,35.453],marker='P',s=170,zorder=21,linewidths=2,edgecolor='w')


# fname = os.path.join('MBON_all_map.png')
# fname = '/Users/patrick/Documents/CeNCOOS/store-locator/data/sanctuary_files/gfnms_py2/GFNMS_py.shp'
# GF_shape = Reader(fname)
# ax.add_geometries(GF_shape.geometries(),ccrs.PlateCarree(),edgecolor='k',lw=2)
# fname = '/Users/patrick/Documents/CeNCOOS/store-locator/data/sanctuary_files/mbnms_py2/mbnms_py.shp'
# GF_shape = Reader(fname)
# ax.add_geometries(GF_shape.geometries(),ccrs.PlateCarree(),facecolor='#c51b8a',alpha=.85,edgecolor="k",lw=2)
# fname = '/Users/patrick/Documents/CeNCOOS/store-locator/data/sanctuary_files/cbnms_py2/CBNMS_py.shp'
# GF_shape = Reader(fname)
# ax.add_geometries(GF_shape.geometries(),ccrs.PlateCarree(),facecolor='#756bb1',alpha=.85,edgecolor="k",lw=2)

plt.savefig("MBON_all_map.png", dpi = 300, bbox_inches='tight', pad_inches=.5)

In [ ]:
rf_2015['station'].unique()

### SST Data

In [ ]:
# ghrsst_NAVO-L4HR1m-GLOB-K10_SST
url="https://thredds.jpl.nasa.gov/thredds/dodsC/ncml_aggregation/OceanTemperature/ghrsst/aggregate__ghrsst_NAVO-L4HR1m-GLOB-K10_SST.ncml"
sst_ds = xr.open_dataset(url)
sst_recent = sst_ds.sel(time=slice(dt.datetime(2018,5,1),dt.datetime(2018,6,1)), lat=slice(42, 31), lon=slice(-130.5,-116.75)) #extent=(-130.5,-116.75,31,42))
xx,yy = np.meshgrid(sst_recent['lon'], sst_recent['lat'])

In [ ]:
fig, ax = make_map()
im = ax.pcolormesh(xx, yy, sst_recent['analysed_sst'].mean(dim='time')-273.15, cmap=cm.thermal, vmin=10, vmax=18)
plot_seal_tracks(ax, atn_df)
ax.scatter(df_gliders['longitude (degrees_east)'], df_gliders['latitude (degrees_north)'],s=20,marker='.',c='k') # Gliders
ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'],marker="D",edgecolor='k',c='.75') # Rockfish

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=.45, axes_class=plt.Axes)
fig.add_axes(ax_cb)
cbar = plt.colorbar(im, cax=ax_cb,ticks=range(9,21,2))
cbar.ax.tick_params(labelsize=20) 
 
fname = os.path.join('MBON_sst_map.png')
# plt.savefig(fname, dpi = 300, bbox_inches='tight', pad_inches=.5)

## Georeference seascape raster

In [ ]:
fig, ax = make_map()

img_extent = [-135, -90, 0, 60]
fname = "./MAY_CCS_seascape.jpg"

img = plt.imread(fname)
ax.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
plot_seal_tracks(ax, atn_df)
ax.scatter(df_gliders['longitude (degrees_east)'], df_gliders['latitude (degrees_north)'],s=20,marker='.',c='k') # Gliders
ax.scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'],marker="D",edgecolor='k',c='.75') # Rockfish


fname = os.path.join('MBON_seascape_map.png')
plt.savefig(fname, dpi = 300, bbox_inches='tight', pad_inches=.5)

### Make Panel Maps

In [ ]:
# Mapping helper function
def make_map_panel(projection=ccrs.PlateCarree(),extent=(-130.5,-116.75,31,42),n=4,sharex=True):
    fig, ax = plt.subplots(nrows=2,ncols=2,
        subplot_kw={'projection': projection}
    )
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.05, hspace=.05)
    fig.set_size_inches(7,6)

    for i, ax_i in enumerate(ax.flatten()):
        ax_i.set_extent(extent, crs=ccrs.PlateCarree())
        gl = ax_i.gridlines(draw_labels=True,)
        gl.xlines = False
        gl.ylines = False
        gl.ylabels_right = False
        ax_i.coastlines('50m')
        ocean_50m  = cfeature.NaturalEarthFeature(category='physical', name='ocean',
                                scale='50m', facecolor=cfeature.COLORS['water'])
        ax_i.add_feature(ocean_50m,zorder=0)
        plt.xticks(rotation='vertical')
        gl.xlocator = mticker.FixedLocator(np.arange(-129,-117,3))
        gl.ylocator = mticker.FixedLocator(np.arange(32,42,3))
#         gl.xformatter = LONGITUDE_FORMATTER
#         gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 16}
        gl.ylabel_style = {'size': 16}
        if i == 0:
            gl.xlabels_bottom= False
            gl.xlabels_top= False
        elif i == 1:
            gl.ylabels_left = False
            gl.xlabels_bottom= False
            gl.xlabels_top= False
        elif i == 2:
            gl.ylabels_right= False
            gl.xlabels_top= False
        else:
            gl.ylabels_left= False
            gl.ylabels_right= False
            gl.xlabels_top= False

    return fig, ax.flatten()

# fig, ax = make_map_panel()

In [ ]:
fig, ax = make_map_panel()
ax[0].quiver(xx[ix],yy[ix],u.values[ix],v.values[ix],speed[ix],
          cmap=cm.speed,
          scale_units='xy',
          scale=.75) # HFR
ax[1].scatter(df_gliders['longitude (degrees_east)'], df_gliders['latitude (degrees_north)'],s=5,marker='.',c='k') # Gliders
ax[2].scatter(rf_2015['longitude (degrees_east)'], rf_2015['latitude (degrees_north)'], marker=".",c='k',s=3) # Rockfish
plot_seal_tracks(ax[3], atn_df)

fname = os.path.join('MBON_panel_map.png')
plt.savefig(fname, dpi = 300, bbox_inches='tight', pad_inches=.5)